# Credo Governance Integration

This notebook is a comprehensive overview of how you can interact with the Credo AI Governance Platform.

All the code shown here can be found on [github](https://github.com/credo-ai/credoai_lens/blob/develop/docs/notebooks/governance_integration.ipynb).


Connecting Lens to the Governance App requires that you have already defined a Use-Case and registered a model to that use-case.

## Glossary
A list of useful concepts to keep in mind while
- **Credo AI Platform**: Also referred to as simply "platform". The central AI governance/reporting Platform, found at [https://app.credo.ai/](https://app.credo.ai/)
- **credoconfig**: configuration file to be copied in the user's home folder
- **use_case_name**: The name of your Use Case as it is registered on Credo AI Platform
- **policy_pack**: A set of responsible AI controls that a use case needs to satisfy. A use case can have multiple policy packs applied to it.
- **policy_pack_key**: A unique identifier for a policy pack (where do we get this?)
- **assessment_plan**: A set of evaluations that `Lens` needs to run in order to satisfy a policy pack requirements.
- **evidence**: The output of a `Lens` evaluation, formatted specifically to be uploaded to the platform.


## 1. Setup 

### Get a config file
This file contains all the necessary information to connect Lens to the Credo AI Platform.  

The default location where Lens will look for it is your home directory (`~/.credoconfig`). The structure of the config should look like this

```
TENANT=<tenant name> # Example: credoai
CREDO_URL=<your credo url> 
API_KEY=<your api key> # Example: JSMmd26...
```

To generate the config file, once you logged into the platform, click on your name icon (top left) and follow:  
My Settings -> Tokens -> Plus Sign -> Generate  
Immediately after generating the token, you will be given the possibility to download the config file.

### Get your model ready
Since a use case and model were already registered on the Platform already, we assume that a fitted model is available. In this tutorial we will emulate the modeling phase by running a quick script.

In [1]:
# model and df are defined by this script
%run training_script.py

## 2. Create Governance
In this phase we create the Governance object with all the necessary information required to run a battery of assessments defined by your use case and policy pack.

In [2]:
from credoai.governance import Governance
gov = Governance()

After initializing the `Governance` object, we need to register an assessment plan. This can be achieved in three ways:
1. Using the `assessment_plan_url`. In the Platform: your_use_case -> Reports -> Your_policy_pack -> **ASSESSMENT PLAN** -> **Copy URL**
2. Using the assessment plan json. Follow steps in 1., but click on **Download** as the last step. (more details in [Air Gap Environment](##Air-Gap-Environment))
3. Using use_case_name and policy pack key as they are defined in the Platform

In the tutorial we will follow 1. 


In [ ]:
url = 'your_assessment_url'
gov.register(assessment_plan_url=url)

# Alternatively method 2.
# gov.register(use_case_name='your_use_case_name', policy_pack_key='your_policy_pack_id')

## 3. Run Lens
This part is very similar to a standard `Lens` run. See the quickstart notebook !!insert link!! for more general information on `Lens`.

###  Create artifacts

In [11]:
from credoai.artifacts import ClassificationModel, TabularData

credo_model = ClassificationModel("credit_default_classifier", model)
test_data = TabularData(
    name="UCI-credit-default-test",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

### Run 

In [12]:
from credoai.lens import Lens
lens = Lens(
    model=credo_model,
    assessment_data=test_data,
    governance=gov
)

Since `Governance` is included in the initialization of Lens, there is no need to specify which evaluators to include in the pipeline. Under the hood, `Lens` converts the assessment plan contained in `gov` into a list of evaluators.

A simple `lens.run()` is sufficient to run all the required assessments.

In [ ]:
lens.run()

If the execution proceeded according to expectations, the evidences, i.e., the results of the assessments, can be sent to the `Governance` object and finally exported to the platform.

In [ ]:
lens.send_to_governance(True)
gov.export()

That's it! With just a few lines of code, a set of evidences documenting compliance to a set of controls defined by a policy pack has been uploaded to the Platform.

## Air Gap Environment

If you cannot interact with Credo AI's Governance App via the internet, this section is for you!

Instead of Lens automating the communication between your assessment environment and governance, you will instead have to download an assessment plan, provide it to `Governance`, run `Lens` and finally upload the evidences (assessments results) to the platform.



### Getting the assessment spec
The asset that you have to get _from_ the governance app is the assessment plan. This can be found in:   

your_use_case -> Reports -> Your_policy_pack -> **ASSESSMENT PLAN** -> **Download**  

Once you download the assessments plan, you can pass it to the Governance object, see code below.


In [ ]:
gov.register(assessment_plan_file='your_assessment_file.json')

Alternatively, you can also register the json in string form using:

In [ ]:
gov.register(assessment_plan='your_long_json_string_assessment_plan')



##### ???Is this still true???
**Important Note On Data Registration in an air-gapped environment** 
When interacting with Credo AI's API you _only_ have to register a Use Case and Model within the platform - nothing needs to be done relating to data. Lens will handle registering your training/assessment data to the app 
However, when in an air-gapped environment you WILL need to register the training and validation data in the platform first. Otherwise Lens will fail.


### Exporting Results

Lens can export evidences to file as a json object. This is generally useful if you want to store your results locally, but particularly useful if you are in an air gapped environment where you cannot access Credo AI's Governance Platform directly from your development environment.

Doing this only requires a one line change in our current flow. We change the `export` argument from `Governance` to a local path:


In [20]:
# To save the evidences locally
gov.export('evidences_test.json')

2022-09-27 17:13:43,628 - lens - INFO - It is not registered, please register first


False

Running the above will create a folder that you specified, if it doesn't exist. It will then create a json file with the assessment results, which can be uploaded to Credo AI's Governance Platform. This json file can be uploaded by going to:  

your_use_case -> Reports -> Your_policy_pack -> **UPLOAD LENS EVIDENCE**